# Video

### FFMPEG

First, check if you already have ffmpeg installed:

In [ ]:
!ffmpeg

If you don't have it installed, you can use:  

`!conda install -c conda-forge ffmpeg` in a new cell.  

For Mac you can also use [Homebrew](https://brew.sh/) to install ffmpeg system wide.  
Simply `!brew install ffmpeg`

You can check your installation by running:

In [ ]:
!ffmpeg

Next, you can install the ffmpeg bindings for python:

In [ ]:
!pip install ffmpeg-python

Let's make a folder called videos:

In [ ]:
import os

out_dir = "videos"

if not os.path.exists(out_dir):
    
    os.makedirs(out_dir)

We can use yt-dlp to download a video from youtube:

In [ ]:
import yt_dlp

url = "https://www.youtube.com/watch?v=cu5ETFpQItY"

ydl_opts = {
    'outtmpl': out_dir + '/%(id)s.%(ext)s',
    'quiet': True,
    'format_sort': ['res:1080', 'ext:mp4:m4a'],
    'ignoreerrors': False
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    
    ydl.download(url)  

Print the name of the video we just downloaded:

In [ ]:
ls videos

In [ ]:
video_path = "videos/cu5ETFpQItY.mp4"

In [ ]:
from IPython.display import Video

Video(video_path, width=600)

Now we can import the ffmpeg python bindings:

In [ ]:
import ffmpeg

#### Some basic operations

In [ ]:
input = ffmpeg.input(video_path)

# The echo filter: http://underpop.online.fr/f/ffmpeg/help/aecho.htm.gz
audio = input.audio.filter("aecho", 0.8, 0.88, 600, 0.4) 
video = input.video.vflip()
out = ffmpeg.output(audio, video, 'videos/flipped_echo.mp4')

out.overwrite_output().run()

In [ ]:
Video('videos/flipped_echo.mp4', width=600)

#### Use ffprobe to access information of any file

In [ ]:
probe = ffmpeg.probe(video_path)

vid_probe = next((stream for stream in probe['streams'] if stream['codec_type'] == 'video'), None)

if vid_probe is None:

    print('[ERROR]: No video stream found', file=sys.stderr)
    
else:
    
    width = int(vid_probe['width'])
    height = int(vid_probe['height'])
    codec = str(vid_probe['codec_name'])
    duration = float(vid_probe['duration'])
    frame_rate = str(vid_probe['r_frame_rate'])
    
print("width: {}, height: {}, codec: {}, duration: {}, frame rate: {}".format(width, height, codec, duration, frame_rate))
    

#### Extract a clip at a specific time

In [ ]:
stream = (
    ffmpeg
    .input(
        video_path,
        ss = 2.0, # where to seek the video, in seconds
        t = 1.0 # duration of clip, in seconds
    )
)

out_path = 'videos/short_clip.mp4'

ffmpeg.output(stream, stream.audio, out_path).global_args('-loglevel', 'quiet').overwrite_output().run()

In [ ]:
from IPython.display import Video

Video(out_path, width=600)

#### Use numpy to transform each frame

In [ ]:
import numpy as np

In [ ]:
in_path = "videos/cu5ETFpQItY.mp4"

out, err = (
    ffmpeg.input(in_path)
    .output('pipe:', format='rawvideo', pix_fmt='rgb24')
    .run(capture_stdout=True)
)

out_path = "videos/numpy.mkv"

process2 = (
    ffmpeg
    .input('pipe:', format='rawvideo', pix_fmt='rgb24', s='{}x{}'.format(width, height))
    .output(out_path, pix_fmt='yuv420p')
    .overwrite_output()
    .run_async(pipe_stdin=True)
)

video = (
    np
    .frombuffer(out, np.uint8)
    .reshape([-1, height, width, 3])
)

for frame in video:

    inverted = 255 - frame
    
    process2.stdin.write(
        inverted
        .astype(np.uint8)
        .tobytes()
    )
        
process2.stdin.close()

#### Convert mkv to mp4 and use the original audio

In [ ]:
input_audio = ffmpeg.input(in_path).audio

out_path = "videos/numpy.mp4"

input_video = ffmpeg.input('videos/numpy.mkv')

(
    ffmpeg
    .output(input_video.video, input_audio, out_path, vcodec="h264", acodec="aac", pix_fmt='yuv420p')
    .overwrite_output()
    .run()
)

In [ ]:
from IPython.display import Video

Video(out_path, width=600)

Tmix is a filter to mix successive video frames.

It accepts the following options:

frames
The number of successive frames to mix. If unspecified, it defaults to 3.

weights
Specify weight of each input video frame. Each weight is separated by space. If number of weights is smaller than number of frames last specified weight will be used for all remaining unset weights.

scale
Specify scale, if it is set it will be multiplied with sum of each weight multiplied with pixel values to give final destination pixel value. By default scale is auto scaled to sum of weights.

In [ ]:
in_path = "videos/cu5ETFpQItY.mp4"
out_path = "videos/cool_effect.mp4"

conversion = (
    ffmpeg
    .input(in_path)
    .filter("tmix", frames=8, weights="1 1 1 1 1 1 1 1")
    .output(out_path, vcodec="h264", pix_fmt='yuv420p')
    .overwrite_output()
    .run()

)

In [ ]:
from IPython.display import Video

Video(out_path, width=600)

#### Display ffmpeg in jupyter

In [ ]:
!pip install ipywidgets

Note: in order for widgets to work you might have to enter this command in your terminal and restart the jupyter lab:  
`jupyter nbextension enable --py --sys-prefix widgetsnbextension`

In [ ]:
from ipywidgets import interact
from matplotlib import pyplot as plt
import ipywidgets as widgets

in_path = "videos/cu5ETFpQItY.mp4"
probe = ffmpeg.probe(in_path)
video_info = next(s for s in probe['streams'] if s['codec_type'] == 'video')
width = int(video_info['width'])
height = int(video_info['height'])
num_frames = int(video_info['nb_frames'])

out, err = (
    ffmpeg
    .input(in_path)
    .output('pipe:', format='rawvideo', pix_fmt='rgb24')
    .global_args('-loglevel', 'quiet')
    .run(capture_stdout=True)
)
video = (
    np
    .frombuffer(out, np.uint8)
    .reshape([-1, height, width, 3])
)

@interact(frame=(0, num_frames))
def show_frame(frame=0):
    plt.imshow(video[frame,:,:,:])

https://trac.ffmpeg.org/wiki/FancyFilteringExamples

### Mixing several videos

In [ ]:
video_dir = "videos"

for file in os.listdir(video_dir):

        if file.endswith(".mp4"):

            file_path = os.path.join(video_dir, file)

            out_path = os.path.join(out_dir, file)

            probe = probe_video(file_path)
            
            
            stream = (
                ffmpeg
                .input(file_path, ss=0, t=MIN_LENGTH)
            )

            video = (
                stream
                .video
                .filter('fps', fps=25, round='down')
                .filter('scale', width=512, height=910, force_original_aspect_ratio='increase')
                .crop(x=0, y=0, width=512, height=910)
                .filter('setsar', r=1)
                .filter('format', pix_fmts='yuv420p')
                .filter('settb', tb='AVTB')
                .filter('setpts', expr='PTS-STARTPTS')
            )

            audio = (
                stream
                .audio
            )

            out = (
                ffmpeg
                .output(video, audio, out_path, vcodec="h264", acodec="aac", pix_fmt='yuv420p')
                .overwrite_output()
                .run()
            )

In [ ]:
import random

streams = []

for file in os.listdir(out_dir):

    file_path = os.path.join(out_dir, file)

    stream = (
        ffmpeg
        .input(file_path)
    )

    streams.append(stream)

    print("adding video", file)

random.shuffle(streams)

In [ ]:
# glow grainextract xor grainmerge negation phoenix average interpolate softlight
blend_mode = "phoenix"

a = streams[0].video.filter('format', pix_fmts='gbrp')
b = streams[1].video.filter('format', pix_fmts='gbrp')
c = streams[2].video.filter('format', pix_fmts='gbrp')

vout = ffmpeg.filter([a, b], 'blend', all_mode=blend_mode)
#
vout = ffmpeg.filter([vout, c], 'blend', all_mode=blend_mode).filter('eq', brightness=0, gamma_weight=-1.0, saturation=0.5)

vout = ffmpeg.filter(vout, 'format', pix_fmts='yuv420p')

audio = ffmpeg.filter(
    [
        streams[0].audio,
        streams[1].audio,
        streams[2].audio
    ],
    'amix',
    inputs=3
)

blend = (
    ffmpeg
    .output(vout, audio, 'test.mp4', vcodec="h264", acodec="aac",  pix_fmt='yuv420p', format='mp4', r=25)
    .overwrite_output()
    .run()
)

## Using python to create supercuts with subtitle files

https://lav.io/2014/06/videogrep-automatic-supercuts-with-python/   

https://github.com/antiboredom/videogrep  

https://www.youtube.com/embed/qEtEbXVbYJQ?feature=oembed  

In [ ]:
!pip install videogrep

In [ ]:
import yt_dlp

out_dir = "videos"

url = "https://www.youtube.com/watch?v=gElfIo6uw4g"

ydl_opts = {
    'outtmpl': out_dir + '/%(id)s.%(ext)s',
    'quiet': False,
    'format_sort': ['res:1080', 'ext:mp4:m4a'],
    'writeautomaticsub': True
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    
    ydl.download(url) 

In [ ]:
ls videos

In [ ]:
mv videos/gElfIo6uw4g.en.vtt videos/gElfIo6uw4g.vtt

In [ ]:
from videogrep import videogrep

# videogrep('videos/gElfIo6uw4g.mp4', 'experience', 'fragment', 'videos/out.mp4')

In [ ]:
!videogrep --input videos/gElfIo6uw4g.mp4 --search 'experience' --search-type fragment --output videos/experience.mp4

In [ ]:
from IPython.display import Video

Video("videos/experience.mp4", width=600)